### Dataset - Load Data

Start by importing the simulator data from the training_data directory.

In [117]:
import csv
import os
import numpy as np

def read_csv(filepath, num_features=7, delimiter=';'):
    data_array = np.array(np.zeros(shape=num_features), ndmin=2)    
    with open(filepath, newline='') as csvfile:
        annotations_reader = csv.reader(csvfile, delimiter=delimiter, quotechar='|')
        for row in annotations_reader:
            data_array = np.vstack((data_array, np.array(row, ndmin=2)))
    return data_array[2:]

In [137]:
DRIVING_LOG_CSV = 'driving_log.csv'
annotations = read_csv(os.path.join('merged_data', DRIVING_LOG_CSV), delimiter=',')
center_image_paths, steering_angles = annotations[:,0], annotations[:,3]
left_image_paths, right_image_paths = annotations[:,1], annotations[:,2]

print('center_image_paths shape', center_image_paths.shape)
print('left_image_paths shape', left_image_paths.shape)
print('right_image_paths shape', right_image_paths.shape)
print('steering_angles shape', steering_angles.shape)

center_image_paths shape (255,)
left_image_paths shape (255,)
right_image_paths shape (255,)
steering_angles shape (255,)


In [135]:
import cv2
TRAINING_DATA_DIR = 'merged_data'
def load_images(image_paths):
    images = np.array(np.zeros(shape=(1, 160, 320, 3)), ndmin=2)
    for path in image_paths:
        image = np.array(cv2.imread(os.path.join(TRAINING_DATA_DIR,path)), ndmin=4)
        images = np.vstack((images, image))
    return images

center_images = load_images(center_image_paths)

256


In [136]:
import pickle
import math


center_images.shape


(256, 160, 320, 3)

In [46]:
with open('train.p', 'rb') as f:
    data = pickle.load(f)

# TODO: Load the feature data to the variable X_train

X_train, y_train = data['features'], data['labels']

# TODO: Load the label data to the variable y_train

In [47]:
# # STOP: Do not change the tests below. Your implementation should pass these tests. 
# assert np.array_equal(X_train, data['features']), 'X_train not set to data[\'features\'].'
# assert np.array_equal(y_train, data['labels']), 'y_train not set to data[\'labels\'].'
# print('Tests passed.')

## Preprocess the Data

1. Shuffle the data
2. Normalize the features using Min-Max scaling between -0.5 and 0.5
3. One-Hot Encode the labels

### Shuffle the data
Hint: You can use the [scikit-learn shuffle](http://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html) function to shuffle the data.

In [48]:
# TODO: Shuffle the data
perm = np.arange(X_train.shape[0])
np.random.shuffle(perm)
X_train = X_train[perm]
y_train = y_train[perm]

In [49]:
# # STOP: Do not change the tests below. Your implementation should pass these tests. 
# assert X_train.shape == data['features'].shape, 'X_train has changed shape. The shape shouldn\'t change when shuffling.'
# assert y_train.shape == data['labels'].shape, 'y_train has changed shape. The shape shouldn\'t change when shuffling.'
# assert not np.array_equal(X_train, data['features']), 'X_train not shuffled.'
# assert not np.array_equal(y_train, data['labels']), 'y_train not shuffled.'
# print('Tests passed.')

### Normalize the features
Hint: You solved this in [TensorFlow lab](https://github.com/udacity/CarND-TensorFlow-Lab/blob/master/lab.ipynb) Problem 1.

In [50]:
# TODO: Normalize the data features to the variable X_normalized
def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [-0.5,0.5]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a=-0.5
    b=0.5
    grayscale_min = 0
    grayscale_max = 255
    return a + (((image_data - grayscale_min)*(b-a))/(grayscale_max - grayscale_min))

X_normalized = normalize_grayscale(X_train)
print('X_normalized shape', X_normalized.shape)
print('X_test shape', X_test.shape)

X_normalized shape (39209, 32, 32, 3)
X_test shape (10000, 32, 32, 3)


In [52]:
# # STOP: Do not change the tests below. Your implementation should pass these tests. 
# assert math.isclose(np.min(X_normalized), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_normalized), 0.5, abs_tol=1e-5), 'The range of the training data is: {} to {}.  It must be -0.5 to 0.5'.format(np.min(X_normalized), np.max(X_normalized))
# print('Tests passed.')

## Train the Network

1. Compile the network using adam optimizer and categorical_crossentropy loss function.
2. Train the network for ten epochs and validate with 20% of the training data.

In [64]:
from keras.models import Graph, Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize, imsave
import cv2
import sys
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

batch_size = 128
nb_classes = 43
nb_epoch = 50
#data_augmentation = True

# Using SqueezeNet
model = Graph()
model.add_input(name='input',input_shape=(32,32,3))

#conv_1: 
model.add_node(Convolution2D(96, 3, 3, activation='relu', init='glorot_uniform',
                             subsample=(2,2),border_mode='valid'),name='conv_1', input='input')

#maxpool_1
model.add_node(MaxPooling2D(pool_size=(3,3),strides=(2,2)),name='maxpool_1', input='conv_1')

#fire module 1
model.add_node(Convolution2D(16, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire2_squeeze', input='maxpool_1')
model.add_node(Convolution2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire2_expand1', input='fire2_squeeze')
model.add_node(Convolution2D(64, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire2_expand2', input='fire2_squeeze')
model.add_node(Activation("linear"),name='fire_2', inputs=["fire2_expand1","fire2_expand2"], merge_mode="concat", concat_axis=1)

#fire module 2
model.add_node(Convolution2D(16, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire3_squeeze', input='fire_2')
model.add_node(Convolution2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire3_expand1', input='fire3_squeeze')
model.add_node(Convolution2D(64, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire3_expand2', input='fire3_squeeze')
model.add_node(Activation("linear"),name='fire_3', inputs=["fire3_expand1","fire3_expand2"], merge_mode="concat", concat_axis=1)

# #fire module  3
# model.add_node(Convolution2D(32, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire4_squeeze', input='fire_3')
# model.add_node(Convolution2D(128, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire4_expand1', input='fire4_squeeze')
# model.add_node(Convolution2D(128, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire4_expand2', input='fire4_squeeze')
# model.add_node(Activation("linear"),name='fire_4', inputs=["fire4_expand1","fire4_expand2"], merge_mode="concat", concat_axis=1)

# #maxpool 4
# model.add_node(MaxPooling2D((2,2)),name='maxpool_4', input='fire_4')

# #fire module  5
# model.add_node(Convolution2D(32, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire5_squeeze', input='maxpool_4')
# model.add_node(Convolution2D(128, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire5_expand1', input='fire5_squeeze')
# model.add_node(Convolution2D(128, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire5_expand2', input='fire5_squeeze')
# model.add_node(Activation("linear"),name='fire_5', inputs=["fire5_expand1","fire5_expand2"], merge_mode="concat", concat_axis=1)

# #fire module 6
# model.add_node(Convolution2D(48, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire6_squeeze', input='fire_5')
# model.add_node(Convolution2D(192, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire6_expand1', input='fire6_squeeze')
# model.add_node(Convolution2D(192, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire6_expand2', input='fire6_squeeze')
# model.add_node(Activation("linear"),name='fire_6', inputs=["fire6_expand1","fire6_expand2"], merge_mode="concat", concat_axis=1)

# #fire module 7
# model.add_node(Convolution2D(48, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire7_squeeze', input='fire_6')
# model.add_node(Convolution2D(192, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire7_expand1', input='fire7_squeeze')
# model.add_node(Convolution2D(192, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire7_expand2', input='fire7_squeeze')
# model.add_node(Activation("linear"),name='fire_7', inputs=["fire7_expand1","fire7_expand2"], merge_mode="concat", concat_axis=1)

# #fire module 8
# model.add_node(Convolution2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire8_squeeze', input='fire_7')
# model.add_node(Convolution2D(256, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire8_expand1', input='fire8_squeeze')
# model.add_node(Convolution2D(256, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire8_expand2', input='fire8_squeeze')
# model.add_node(Activation("linear"),name='fire_8', inputs=["fire8_expand1","fire8_expand2"], merge_mode="concat", concat_axis=1)

# #maxpool 8
# model.add_node(MaxPooling2D((2,2)),name='maxpool_8', input='fire_8')

# #fire module 9
# model.add_node(Convolution2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire9_squeeze', input='maxpool_8')
# model.add_node(Convolution2D(256, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire9_expand1', input='fire9_squeeze')
# model.add_node(Convolution2D(256, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire9_expand2', input='fire9_squeeze')
# model.add_node(Activation("linear"),name='fire_9', inputs=["fire9_expand1","fire9_expand2"], merge_mode="concat", concat_axis=1)

model.add_node(Dropout(0.5),input='fire_3',name='fire9_dropout')

#conv_10
model.add_node(Convolution2D(nb_classes, 1, 1, activation='relu', init='glorot_uniform',border_mode='valid'),
               name='conv_10', input='fire9_dropout')

#avgpool_10 - Global Average Pooling
model.add_node(GlobalAveragePooling2D((13,13)),name='avgpool_10', input='conv_10')

model.add_node(Flatten(),name='flatten',input='avgpool_10')
model.add_node(Dense(nb_classes, activation='softmax'),input='flatten', name='fc_final')

model.add_node(Activation("softmax"),input='fc_final',name='softmax')
model.add_output(name='output',input='softmax')

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

#model.compile(optimizer=adam, loss={'output':'categorical_crossentropy'})

model.compile(loss={'output':'categorical_crossentropy'},
             optimizer=adam,
             metrics=['accuracy'])

print(model.summary())

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
#Y_test = np_utils.to_categorical(y_test, nb_classes)

print('Not using data augmentation...')

callback1 = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, mode='auto')

model.fit({'input':X_normalized,'output':Y_train}, batch_size=batch_size,
         nb_epoch=nb_epoch, validation_split=0.222, shuffle=True,callbacks=[callback1])

# model.fit({'input':X_normalized,'output':Y_train}, batch_size=batch_size,
#           nb_epoch=nb_epoch, validation_data={'input':X_test,'output':Y_test}, shuffle=True,callbacks=[callback1])

(13, 13)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv_1 (Convolution2D)           (None, 15, 15, 96)    2688        input[0][0]                      
____________________________________________________________________________________________________
maxpool_1 (MaxPooling2D)         (None, 7, 7, 96)      0           conv_1[0][0]                     
____________________________________________________________________________________________________
fire2_squeeze (Convolution2D)    (None, 7, 7, 16)      1552        maxpool_1[0][0]                  
__________________________________________________________________________________

KeyboardInterrupt: 

## Build a Multi-Layer Feedforward Network

Build a multi-layer feedforward neural network to classify the traffic sign images.

1. Set the first layer to a `Flatten` layer with the `input_shape` set to (32, 32, 3)
2. Set the second layer to `Dense` layer width to 128 output. 
3. Use a ReLU activation function after the second layer.
4. Set the output layer width to 43, since there are 43 classes in the dataset.
5. Use a softmax activation function after the output layer.

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

### One-Hot Encode the labels
Hint: You can use the [scikit-learn LabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) function to one-hot encode the labels.

In [10]:
# # STOP: Do not change the tests below. Your implementation should pass these tests. 
# import collections

# assert y_one_hot.shape == (39209, 43), 'y_one_hot is not the correct shape.  It\'s {}, it should be (39209, 43)'.format(y_one_hot.shape)
# assert next((False for y in y_one_hot if collections.Counter(y) != {0: 42, 1: 1}), True), 'y_one_hot not one-hot encoded.'
# print('Tests passed.')

In [41]:
# TODO: One Hot encode the labels to the variable y_one_hot
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()

y_one_hot = label_binarizer.fit_transform(y_train)

In [11]:
from keras.layers.core import Flatten, Dense, Activation
from keras.models import Sequential
model = Sequential()
# TODO: Build a Multi-layer feedforward neural network with Keras here.
model.add(Flatten(input_shape=(32,32,3)))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [12]:
# # STOP: Do not change the tests below. Your implementation should pass these tests.
# from keras.layers.core import Dense, Activation, Flatten
# from keras.activations import relu, softmax

# def check_layers(layers, true_layers):
#     assert len(true_layers) != 0, 'No layers found'
#     for layer_i in range(len(layers)):
#         assert isinstance(true_layers[layer_i], layers[layer_i]), 'Layer {} is not a {} layer'.format(layer_i+1, layers[layer_i].__name__)
#     assert len(true_layers) == len(layers), '{} layers found, should be {} layers'.format(len(true_layers), len(layers))

# check_layers([Flatten, Dense, Activation, Dense, Activation], model.layers)

# assert model.layers[0].input_shape == (None, 32, 32, 3), 'First layer input shape is wrong, it should be (32, 32, 3)'
# assert model.layers[1].output_shape == (None, 128), 'Second layer output is wrong, it should be (128)'
# assert model.layers[2].activation == relu, 'Third layer not a relu activation layer'
# assert model.layers[3].output_shape == (None, 43), 'Fourth layer output is wrong, it should be (43)'
# assert model.layers[4].activation == softmax, 'Fifth layer not a softmax activation layer'
# print('Tests passed.')

In [13]:
# TODO: Compile and train the model here.
from keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_normalized, y_one_hot, nb_epoch=6, validation_split=0.2)


Train on 40000 samples, validate on 10000 samples
Epoch 1/6
40000/40000 [==============================] - 18s - loss: 1.7018 - acc: 0.4076 - val_loss: 1.5596 - val_acc: 0.4552
Epoch 2/6
40000/40000 [==============================] - 19s - loss: 1.4755 - acc: 0.4859 - val_loss: 1.5052 - val_acc: 0.4746
Epoch 3/6
40000/40000 [==============================] - 21s - loss: 1.3888 - acc: 0.5144 - val_loss: 1.4721 - val_acc: 0.4931
Epoch 4/6
40000/40000 [==============================] - 20s - loss: 1.3176 - acc: 0.5417 - val_loss: 1.4398 - val_acc: 0.5006
Epoch 5/6
40000/40000 [==============================] - 20s - loss: 1.2611 - acc: 0.5591 - val_loss: 1.4610 - val_acc: 0.4978
Epoch 6/6
40000/40000 [==============================] - 23s - loss: 1.2142 - acc: 0.5768 - val_loss: 1.5130 - val_acc: 0.4829


In [14]:
# # STOP: Do not change the tests below. Your implementation should pass these tests.
# from keras.optimizers import Adam

# assert model.loss == 'categorical_crossentropy', 'Not using categorical_crossentropy loss function'
# assert isinstance(model.optimizer, Adam), 'Not using adam optimizer'
# assert len(history.history['acc']) == 10, 'You\'re using {} epochs when you need to use 10 epochs.'.format(len(history.history['acc']))

# assert history.history['acc'][-1] > 0.92, 'The training accuracy was: %.3f. It shoud be greater than 0.92' % history.history['acc'][-1]
# assert history.history['val_acc'][-1] > 0.85, 'The validation accuracy is: %.3f. It shoud be greater than 0.85' % history.history['val_acc'][-1]
# print('Tests passed.')

## Convolutions
1. Re-construct the previous network
2. Add a [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters, a 3x3 kernel, and valid padding before the flatten layer.
3. Add a ReLU activation after the convolutional layer.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

In [15]:
# TODO: Re-construct the network and add a convolutional layer before the flatten layer.
from keras.layers.core import Dense, Activation, Flatten
from keras.activations import relu, softmax
from keras.layers import Convolution2D

#nb_classes = 43

nb_classes = 10
nb_epoch = 5

# Image Processing Constants
NB_ROWS, NB_COLS, NB_CHANNELS = (32, 32, 3)
IMG_SHAPE = (NB_ROWS, NB_COLS, NB_CHANNELS) if NB_CHANNELS > 1 else (NB_ROWS, NB_COLS)

# Convolutional HyperParameters
nb_filters = 32
kernel_size = (3,3)

model = Sequential()
#Add Convolution Here
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], 
                        input_shape=IMG_SHAPE, border_mode='valid'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))


In [17]:
# # STOP: Do not change the tests below. Your implementation should pass these tests.
# from keras.layers.core import Dense, Activation, Flatten
# from keras.layers.convolutional import Convolution2D

# check_layers([Convolution2D, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)

# assert model.layers[0].input_shape == (None, 32, 32, 3), 'First layer input shape is wrong, it should be (32, 32, 3)'
# assert model.layers[0].nb_filter == 32, 'Wrong number of filters, it should be 32'
# assert model.layers[0].nb_col == model.layers[0].nb_row == 3, 'Kernel size is wrong, it should be a 3x3'
# assert model.layers[0].border_mode == 'valid', 'Wrong padding, it should be valid'

# model.compile('adam', 'categorical_crossentropy', ['accuracy'])
# history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=2, validation_split=0.2)
# assert(history.history['val_acc'][-1] > 0.91), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
# print('Tests passed.')

## Pooling
1. Re-construct the network
2. Add a 2x2 [max pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

In [18]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
from keras.layers.core import Dense, Activation, Flatten
from keras.activations import relu, softmax
from keras.layers import Convolution2D, MaxPooling2D

nb_classes = 43
nb_epoch = 10

# Image Processing Constants
NB_ROWS, NB_COLS, NB_CHANNELS = (32, 32, 3)
IMG_SHAPE = (NB_ROWS, NB_COLS, NB_CHANNELS) if NB_CHANNELS > 1 else (NB_ROWS, NB_COLS)

# Convolutional HyperParameters
nb_filters = 32
kernel_size = (3,3)

model = Sequential()
#Add Convolution Here
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], 
                        input_shape=IMG_SHAPE, border_mode='valid'))
#Add Pooling Layer Here
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [19]:
# # STOP: Do not change the tests below. Your implementation should pass these tests.
# from keras.layers.core import Dense, Activation, Flatten
# from keras.layers.convolutional import Convolution2D
# from keras.layers.pooling import MaxPooling2D

# check_layers([Convolution2D, MaxPooling2D, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)
# assert model.layers[1].pool_size == (2, 2), 'Second layer must be a max pool layer with pool size of 2x2'

# model.compile('adam', 'categorical_crossentropy', ['accuracy'])
# history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=2, validation_split=0.2)
# assert(history.history['val_acc'][-1] > 0.91), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
# print('Tests passed.')

## Dropout
1. Re-construct the network
2. Add a [dropout](https://keras.io/layers/core/#dropout) layer after the pooling layer. Set the dropout rate to 50%.

In [20]:
# TODO: Re-construct the network and add a convolutional layer before the flatten layer.
from keras.layers.core import Dense, Activation, Flatten
from keras.activations import relu, softmax
from keras.layers import Convolution2D, MaxPooling2D, Dropout

nb_classes = 43
nb_epoch = 10

# Image Processing Constants
NB_ROWS, NB_COLS, NB_CHANNELS = (32, 32, 3)
IMG_SHAPE = (NB_ROWS, NB_COLS, NB_CHANNELS) if NB_CHANNELS > 1 else (NB_ROWS, NB_COLS)

# Convolutional HyperParameters
nb_filters = 32
kernel_size = (3,3)

model = Sequential()
#Add Convolution Here
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], 
                        input_shape=IMG_SHAPE, border_mode='valid'))
#Add Pooling Layer Here
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.50))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

In [21]:
# # STOP: Do not change the tests below. Your implementation should pass these tests.
# from keras.layers.core import Dense, Activation, Flatten, Dropout
# from keras.layers.convolutional import Convolution2D
# from keras.layers.pooling import MaxPooling2D

# check_layers([Convolution2D, MaxPooling2D, Dropout, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)
# assert model.layers[2].p == 0.5, 'Third layer should be a Dropout of 50%'

# model.compile('adam', 'categorical_crossentropy', ['accuracy'])
# history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=2, validation_split=0.2)
# assert(history.history['val_acc'][-1] >= 0.91), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
# print('Tests passed.')

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [22]:
# # TODO: Build a model
# from keras.layers.core import Dense, Activation, Flatten
# from keras.activations import relu, softmax
# from keras.layers import Convolution2D, MaxPooling2D, Dropout

# #model Constants
# nb_epoch = 10
# batch_size = 128
# nb_classes = 43


# # Image Processing Constants
# NB_ROWS, NB_COLS, NB_CHANNELS = (32, 32, 3)
# IMG_SHAPE = (NB_ROWS, NB_COLS, NB_CHANNELS) if NB_CHANNELS > 1 else (NB_ROWS, NB_COLS)

# # Convolutional HyperParameters
# nb_filters = 32
# kernel_size = (3,3)

# model = Sequential()
# model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], 
#                         input_shape=IMG_SHAPE, border_mode='valid'))
# model.add(Activation('relu'))
# model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], 
#                         input_shape=IMG_SHAPE, border_mode='valid'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dense(nb_classes))
# model.add(Activation('softmax'))


# # TODO: Compile and train the model
# model.compile(loss='categorical_crossentropy',
#              optimizer='adadelta',
#              metrics=['accuracy'])

# model.fit(X_normalized, y_one_hot, batch_size=batch_size,
#          nb_epoch=nb_epoch, validation_split=0.222)

**Current Validation Accuracy:** (98.09%)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the [`metrics_names`](https://keras.io/models/model/) property to get the labels.

In [ ]:
# TODO: Load test data
with open('test.p', 'rb') as f:
    data_test = pickle.load(f)

X_test = data_test['features']
y_test = data_test['labels']

# TODO: Preprocess data & one-hot encode the labels
X_normalized_test = normalize_grayscale(X_test)
y_one_hot_test = label_binarizer.fit_transform(y_test)

# TODO: Evaluate model on test data
metrics = model.evaluate(X_normalized_test, y_one_hot_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))



**Test Accuracy:** (92.05%)

## Summary
Keras is a great tool to use to quickly build a neural network and evaluate performance.